In [5]:
import numpy as np 
import pandas as pd 
import statsmodels.api as sm 
import statsmodels.formula.api as smf 
from itertools import combinations 
!conda install -c conda-forge --yes plotnine   
import plotnine as p

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\SANKHA\anaconda3

  added / updated specs:
    - plotnine


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.11.0               |   py38haa244fe_0        16.9 MB  conda-forge
    descartes-1.1.0            |             py_4           7 KB  conda-forge
    mizani-0.7.3               |     pyhd8ed1ab_0          53 KB  conda-forge
    palettable-3.3.0           |             py_0          86 KB  conda-forge
    plotnine-0.8.0             |     pyhd8ed1ab_0         3.8 MB  conda-forge
    python_abi-3.8             |           2_cp38           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:        20.9 MB

The following NEW packages will be INSTALLED:

  descartes          conda-forge/noarch::descartes-1.1.0



==> WARNING: A newer version of conda exists. <==
  current version: 4.10.1
  latest version: 4.11.0

Please update conda by running

    $ conda update -n base -c defaults conda




In [10]:
data_judge = pd.read_stata("D:\Replication_Exercises\Mixtape_IV_JudgeFE_JIVE\Python\judge_fe.dta")
data_judge
data_judge['bailDate'] =  (data_judge['bailDate'] - pd.to_datetime('1970-01-01')).dt.days.values
data_judge

,priorCases,prior_felChar,prior_guilt,fel,mis,sum,F1,F2,F3,F,...,t5,t6,judge_pre_1,judge_pre_2,judge_pre_3,judge_pre_4,judge_pre_5,judge_pre_6,judge_pre_7,judge_pre_8
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,0,0,0,0,0
1,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0,0,0,0,0,0,0,1
2,5.0,4.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1,0,0,0,0,0,0,0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,0,0,0,0,0
4,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331966,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0,0,0,1,0,0,0,0
331967,2.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0,0,0,1,0,0,0,0
331968,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0,0,0,1,0,0,0,0
331969,16.0,6.0,4.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0,0,0,1,0,0,0,0


In [11]:
## Grouped Variables formation, similar to setting up globals in stata 
judges_pre = "+".join(data_judge.columns[data_judge.columns.str.contains('^judge_pre_[1-7]')] ) 
demo = "+".join(['black', 'age','male', 'white']) ## demographic variables 
off = "+".join(['fel','mis','sum','F1','F2','F3','M1','M2','M3','M'])
prior = "+".join(['priorCases','priorWI5','prior_felChar','prior_guilt','onePrior','threePriors'])
control2 = "+".join(['day','day2','bailDate','t1','t2','t3','t4','t5'])


In [12]:
OLSmin_formula = "guilt ~ jail3 +" + control2
OLSmax_formula = "guilt ~ jail3 + possess + robbery + DUI1st + drugSell + aggAss + {demo} + {prior} + {off} + {control2}".format(demo = demo, prior = prior, off = off, control2 = control2)

In [29]:
OLS_min = sm.OLS.from_formula(OLSmin_formula, data = data_judge).fit()
OLS_max = sm.OLS.from_formula(OLSmax_formula, data = data_judge).fit()
print("OLS_Min_Result")
print(OLS_min.summary()) 
print("OLS_Max_Result")
print(OLS_max.summary())

OLS_Min_Result
                            OLS Regression Results                            
Dep. Variable:                  guilt   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     276.5
Date:                Mon, 07 Feb 2022   Prob (F-statistic):               0.00
Time:                        14:03:17   Log-Likelihood:            -2.3966e+05
No. Observations:              331971   AIC:                         4.793e+05
Df Residuals:                  331961   BIC:                         4.795e+05
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.4135      0.128     

In [48]:
## IV2SLS 
from linearmodels import IV2SLS 
iv_min_formula = "guilt ~ {control2} + [jail3 ~ {judges_pre}]".format(control2=control2, judges_pre=judges_pre)
max_formula = """guilt ~ {demo} + possess + {prior} + robbery + {off} + DUI1st + {control2} + drugSell + aggAss +
                    [jail3 ~ {judges_pre}]""".format(demo=demo,
                                                    prior=prior,
                                                    off=off,
                                                    control2=control2,
                                                   judges_pre=judges_pre)
iv_min = IV2SLS.from_formula(iv_min_formula, data = data_judge).fit()
iv_max = IV2SLS.from_formula(iv_max_formula, data = data_judge).fit()
print('IV_Min_Result')
print(iv_min.summary)
print('IV_Max_Result')
print(iv_max.summary)

IV_Min_Result
                          IV-2SLS Estimation Summary                          
Dep. Variable:                  guilt   R-squared:                      0.4849
Estimator:                    IV-2SLS   Adj. R-squared:                 0.4849
No. Observations:              331971   F-statistic:                 3.199e+05
Date:                Mon, Feb 07 2022   P-value (F-stat)                0.0000
Time:                        14:41:48   Distribution:                  chi2(9)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
bailDate      3.2e-05  1.902e-06     16.829     0.0000   2.828e-05   3.573e-05
day        -7.952e-05  2.506e-05    -3

In [ ]:
## For Jack-knife Instrumental Variables Estimation, rpy2 package is required which does not run properly on Windows system 
## The codes in this notebook are mostly from 'Causal Inference: The Mixtape' applications, with slight changes (Visit https://mixtape.scunning.com/index.html)
## References: 1.Angrist, J.D., Imbens, G.W. & Krueger, A.B. (1999),"Jackknife Instrumental Variables Estimation", Journal of Applied Econometrics 14:57-67 
## 2. Cunningham,S., "Causal Inference: The Mixtape" (Chapter 7: Instrumental Variables) (Online Version: https://mixtape.scunning.com/index.html)
## 3. Stevenson, M.T. (2018), "Distortion of Justice: How the Inability to Pay Bail Affects Case Outcomes", The Journal of Law, Economics and Organization, Volume 34, Issue 4, Pages 511-542

## For data, check out github page of mixtape (owned by Scott Cunningham): https://github.com/scunning1975/mixtape 